<a href="https://colab.research.google.com/github/Javsk891/Sistemas-de-recomendacion/blob/main/Sistema_de_recomendaci%C3%B3n_M%C3%BAsica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sitema de recomendación de Música

En este pryecto se utilizarán datos de Last.fm obtenidos desde el siguiente link  https://bradfordtuckfield.com/lastfm-matrix-germany.csv". Este ditio web permite a las personas escuchar musica online. Para este caso, los 'items' en nuestra matriz de interacciones serán los artitas que interpretan las cansiones, un '1' en la matrix de interacciones indicará que el ususario ha escuchado determinado artista, en caso contrario, un '0' en la matrix de interacciones indicará que el usuario no ha escuchado al artista.

Comenzamos por importar los datos y organizarlos dentro de un DataFrame de Pandas

In [7]:
import pandas as pd #Importamos la libraría Pandas que nos permite acceder a los datos
last_fm = pd.read_csv('https://bradfordtuckfield.com/lastfm-matrix-germany.csv') #Creamos la variable 'last_fm' y ponemos nuestros datos allí
print(last_fm.head()) #Visualizamos los datos obtenidos

   user  a perfect circle  abba  ac/dc  adam green  aerosmith  afi  air  \
0     1                 0     0      0           0          0    0    0   
1    33                 0     0      0           1          0    0    0   
2    42                 0     0      0           0          0    0    0   
3    51                 0     0      0           0          0    0    0   
4    62                 0     0      0           0          0    0    0   

   alanis morissette  alexisonfire  ...  timbaland  tom waits  tool  \
0                  0             0  ...          0          0     0   
1                  0             0  ...          0          0     0   
2                  0             0  ...          0          0     0   
3                  0             0  ...          0          0     0   
4                  0             0  ...          0          0     0   

   tori amos  travis  trivium  u2  underoath  volbeat  yann tiersen  
0          0       0        0   0          0        

In [8]:
last_fm.drop(['user'], axis = 1, inplace = True) #Hacemos que la columna 'user' se comvierta en el índice de nuestro dataset

Antes de continuar, nota la diferencia entre la matrix de interacciones del ejemplo más sencillo que se planteó antes dentro de este repositorío y la matriz de interacciones que obtenemos en este caso de estudio, aquí, las columnas corresponden a los items y las filas corresponden a los usuarios.

Entonces, debemos trabajar con una transpuesta de dicha matrix de interacciones para facilitar el procesamiento de los datos

In [12]:
last_fm.shape

(1257, 285)

In [11]:
last_fmt = last_fm.T
last_fmt.shape

(285, 1257)

Observando la forma de nuestra matriz, podemos observar que en este caso vamos a estar trabajando con datos que nos brindan información sobre 1257 usuarios y 285 artitas y sus respectivas interacciones.

#Creación del sistema de recomendación

En lo que sigue reutilizaremos el código que ya hemos desarrollado en el ejemplo sencillo de sistemas de recomendación que se encuntra en este mismo repositorio de GitHub. Para ver el código explicado de una forma más sencilla y clara puede ingresar al ejemplo en donde se explica paso a paso desde el calculo de la similaridad coseno, hasta la creación de cada una de las funciones utilizadas

In [14]:
#Rating de productos
conteo_interacciones = last_fmt.copy()
conteo_interacciones.loc[:, 'conteo'] = conteo_interacciones.sum(axis = 1) #Toma la suma de de los productos seleccionados en cada fila y guarda el resultado en una nueva columna llamada conteo
conteo_interacciones = conteo_interacciones.sort_values(by='conteo', ascending = False) #Organiza las filas de la más a la menos popular
print(list(conteo_interacciones.index)) #Imprimimos el rating de los productos

['linkin park', 'coldplay', 'red hot chili peppers', 'rammstein', 'system of a down', 'metallica', 'die toten hosen', 'billy talent', 'the killers', 'the beatles', 'jack johnson', 'muse', 'beatsteaks', 'nirvana', 'foo fighters', 'radiohead', 'placebo', 'arctic monkeys', 'evanescence', 'bloc party', 'rise against', 'the kooks', 'mando diao', 'the white stripes', 'deichkind', 'incubus', 'farin urlaub', 'in flames', 'clueso', 'peter fox', 'the offspring', 'air', 'subway to sally', 'nightwish', 'the prodigy', 'disturbed', 'green day', 'johnny cash', 'ac/dc', 'kings of leon', 'depeche mode', 'amy winehouse', 'bullet for my valentine', 'in extremo', 'blink-182', 'slipknot', 'death cab for cutie', 'daft punk', 'limp bizkit', 'sum 41', 'schandmaul', 'fall out boy', 'kanye west', 'tenacious d', 'seeed', 'rihanna', 'papa roach', 'portishead', 'rage against the machine', 'franz ferdinand', 'nelly furtado', 'marilyn manson', 'queen', 'bob marley', 'feist', 'queens of the stone age', 'massive attac

In [15]:
###############################################################################
############################# Similitud Coseno ################################
###############################################################################

#Producto Punto
def el_producto_punto(vector1, vector2):
  producto_punto = np.sum([vector1[k]*vector2[k] for k in range(0, len(vector1))])
  return(producto_punto)

In [16]:
#Creando el sistema de recomendaciones
def popularidad(interacciones): #Creamos la función para determinar la popularidad de los productos
  conteo_interacciones = last_fmt.copy() #Creamos una copia de la matriz de interacciones
  conteo_interacciones.loc[:, 'conteo'] = conteo_interacciones.sum(axis = 1)
  ranking = conteo_interacciones.sort_values(by = 'conteo', ascending = False) #Aquí obtenemos el ranking ene orden ascendente de los productos más populares
  mas_populares = list(ranking.set_index)
  return(mas_populares)

In [17]:
#Norma del vector
def vector_norm(vector):
  norma = np.sqrt(el_producto_punto(vector, vector))
  return(norma)

In [18]:
#Similaridad Coseno
def similaridad_coseno(vector1, vector2):
  producto_punto = el_producto_punto(vector1, vector2)
  coseno = producto_punto/(vector_norm(vector1)*vector_norm(vector2))
  coseno = np.round(coseno, 4)
  return(coseno)

In [22]:
articulo_de_interes = 'abba' #Creamos la variable 'articulo_de_interes' para almacenar el item de interes que ya se conoce sobre el nuevo cliente
otros_articulos = [nombre_articulo for nombre_articulo in last_fmt.index if nombre_articulo != articulo_de_interes]
otros_articulos = last_fmt.loc[otros_articulos, :]
el_articulo = last_fmt.loc[articulo_de_interes, :]

In [23]:
similaridades = [] #Creamos una lista vacia en donde se almacenarán los artiuclos similares que deberían recomendarse al nuevo cliente
for articulos in otros_articulos.index:
  similaridades.append(similaridad_coseno(el_articulo, otros_articulos.loc[articulos, :]))

otros_articulos['Similaridades'] = similaridades
recomendaciones = list(otros_articulos.sort_values(by = 'Similaridades', ascending = False).index)

In [27]:
def recomendación_articulos(last_fmt, nombre_articulo):
  otras_filas = [nombre_articulo_rec for nombre_articulo_rec in last_fmt.index if nombre_articulo_rec != nombre_articulo]
  otros_articulos = last_fmt.loc[otras_filas, :]
  el_articulo = list(last_fmt.loc[nombre_articulo, :])
  similaridades = []
  for articulos in otros_articulos.index: # Iteración sobre el índice del dataframe
    similaridades.append(similaridad_coseno(el_articulo, otros_articulos.loc[articulos, :]))
  otros_articulos['Similaridades'] = similaridades
  return list(otros_articulos.sort_values(by = 'Similaridades', ascending = False).index)

Aquí podemos visualizar la lista de los artistas que se recomiendan a un usuario del que se sabe previamente que le gusta el artista 'abba'

In [28]:
print(recomendación_articulos(last_fmt, 'abba'))

['madonna', 'robbie williams', 'elvis presley', 'michael jackson', 'queen', 'the beatles', 'kelly clarkson', 'groove coverage', 'duffy', 'mika', 'rihanna', 'britney spears', 'christina aguilera', 'nightwish', 'amy macdonald', 'atb', 'howard shore', 'frank sinatra', 'coldplay', 'depeche mode', 'avril lavigne', 'hans zimmer', 'scooter', 'deep purple', 'dido', 'u2', 'pink', 'stars', 'kate nash', 'the pussycat dolls', 'dream theater', 'tegan and sara', 'clueso', 'jason mraz', 'james blunt', 'green day', 'norah jones', 'r.e.m.', 'justin timberlake', 'metallica', 'the rolling stones', 'mgmt', 'bruce springsteen', 'mando diao', 'beirut', 'pink floyd', 'katie melua', 'porcupine tree', 'faithless', 'lily allen', 'tori amos', 'aerosmith', 'hot chip', 'genesis', 'beyonce', 'leona lewis', 'nelly furtado', 'linkin park', 'eric clapton', 'david bowie', 'dire straits', 'keane', 'daft punk', 'panic at the disco', 'die toten hosen', 'led zeppelin', 'amy winehouse', 'black sabbath', 'johnny cash', 'ac/d